In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

/home/dat/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# start = 2017
# end = 2021
# url = "https://s.cafef.vn/bao-cao-tai-chinh/G36/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn"
# fails = []
name="BIC"
a = None
url1 = url.replace("/G36/", f"/{name}/")
df = get_head(url1)


NameError: name 'url' is not defined

In [2]:
def get_head(url):
    df = get_data(url, True)
    for i in range(end-1, start-1, -1):
#     for i in range(2017, 2018):
        url1 = url.replace("2021",str(i))
        df1 = get_data(url1)
        df = pd.concat([df1, df], axis=1)
    return df

In [3]:

def get_data(url, first=False):
    print(url)
    url_2 = url.replace("BSheet", "IncSta").replace("bao-cao-tai-chinh-","ket-qua-hoat-dong-kinh-doanh-")
    url_3 = url.replace("BSheet", "CashFlow").replace("bao-cao-tai-chinh-","luu-chuyen-tien-te-gian-tiep-")
    
    data1 = pd.read_html(url)
    # get name colums
    names = data1[2]
    cls_name = names.iloc[0].tolist()
    cls_name[0] = "Tiêu chí"
    cls_name = cls_name[:5]
    
    # get data from 1 to 9
        # get data
    df1 = data1[3]
    df1 = df1.iloc[[0,13,14,15,16,17,21],[0,1,2,3,4]]
    df1.columns = cls_name
    
    # lấy tài sản cố định vô hình
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text

    # Parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    
    # print(soup.prettify()) # print the parsed data of html
    table = soup.find("table", id="tableContent")

    rows = table.find_all('tr')

    tscd = []
    for row in rows:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        if len(cols) > 0 and "Tài sản cố định vô hình" in cols[0]:
            tscd = cols
    a = [int(x.replace(",","")) if len(x) > 0 else 0 for x in tscd[1:5]]
    b = [tscd[0]]
    b = b + a
    s = pd.Series(b, index=df1.columns)
    df1 = df1.append(s, ignore_index=True)
    df1 = df1.iloc[[0,7,1,2,3,4,5,6]]
    
    # get data from 10 to 13
    data2 =  pd.read_html(url_2)
    df2 = data2[3]
    df2 = df2.iloc[[0,2,3,18],[0,1,2,3,4]]
    df2.columns = cls_name
    
    # geta data 14 (25 gian tiep 8 truc tiep)
    data3 =  pd.read_html(url_3)
    df3 = data3[3]
    df3 = df3.iloc[[25],[0,1,2,3,4]]
    df3.columns = cls_name

    df = pd.concat([df1,df2, df3], axis=0)
    df = df.replace(np.nan, 0)
#     print(df)
#     return df
    df.columns = df.columns.str.strip()
    df.iloc[:, [1,2,3,4]] = df.iloc[:, [1,2,3,4]].astype('float')
    
    df = df.reset_index(drop=True)
    
    if first:
        return df
    else:
        return df.iloc[:,[1,2,3,4]]
    


In [4]:
def get_eps(name="G36"):
    url = f"https://data.masvn.com/Controls/Report/StockFinance_Quarter.aspx?language=vi&scode={name}&page=1"
    data = pd.read_html(url)
    df = data[0]
    data_eps = df.iloc[[13,17], [0,2,3,4,5]].reset_index(drop=True)
    data_eps.columns = data_eps.iloc[0]
    data_eps = data_eps.drop([0])
    
    
    for page in range(2,6):
        url = f"https://data.masvn.com/Controls/Report/StockFinance_Quarter.aspx?language=vi&scode={name}&page={page}"
        data = pd.read_html(url)
        df = data[0]
        eps = df.iloc[[13,17], [2,3,4,5]].reset_index(drop=True)
        eps.columns = eps.iloc[0]
        eps = eps.drop([0])
        data_eps = pd.concat([eps, data_eps], axis = 1)
        
    filter_col = [col for col in data_eps if any( k in col for k in ['2017', '2018', '2019', '2020', '2021'])]
    select_col = ["CHỈ TIÊU CƠ BẢN"] + filter_col
    data_eps = data_eps[select_col].reset_index(drop=True)
    return data_eps


In [5]:
def wrap(url, name, index=4):
    print(f"Crawl data name {name}")

    df = get_head(url)
    df = df.loc[:,~df.columns.duplicated()]
    df_eps = get_eps(name)
    df_eps=df_eps.replace('-',0)
    df_eps = df_eps.loc[:,~df_eps.columns.duplicated()]


    for quarter in range(1,5):

        query = str(quarter) + "-"
        filter_col = [col for col in df if query in col ]
        select_col = ["Tiêu chí"] + filter_col
        df_1 = df[select_col]

        s = df_1.iloc[2]
        n = ["Tổng tải sàn bình quân "]
        n.append(s[1])
        for i in range(2, len(s)):
            n.append(int((s[i]+s[i-1]+1)/2))

        new = pd.Series(n, index=df_1.columns)
        df1 = df_1.append(new, ignore_index=True)
        df2 = df1.iloc[[0,1,2,13,3,4,5,6,7,8,9,10,11,12]]
        df2 = df2.reset_index(drop=True)

        query = str(quarter) + "/"
        filter_col = [col for col in df_eps if query in col ]
        select_col = ["CHỈ TIÊU CƠ BẢN"] + filter_col
        df3 = df_eps[select_col].reset_index(drop=True)
    

        eps = [df3.iloc[0,0]]
        year = 2016
        for i in range(1,len(df3.columns)):
            year = year + 1
            while str(year) not in df3.columns[i] and year < 2022:
                eps.append(0)
                year = year + 1

            eps.append(int(df3.iloc[0,i]))
    
        eps = eps + [0] * 5
        k = len(df2.columns)
        eps = eps[0:k]

        df2 = df2.append(pd.Series(eps, index=df2.columns), ignore_index=True)
    
        df2.to_csv(f"data_{index}/{name}/{name}.csv", mode='a', index_label="stt")
#         my_file = Path("output.xlsx")
#         if my_file.is_file():
#             with pd.ExcelWriter('output.xlsx', mode='a') as writer:
#                 data.to_excel(writer, sheet_name=name)
#         else:
#             data.to_excel("output.xlsx", sheet_name=name)
#     return data

In [6]:
def get_company_name(url):

    html_content = requests.get(url).text

    # Parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    name_company = soup.find("title").text.strip()
    for r in (("Báo cáo tài chính", ""), ("|", ""), (": ",""), ("CafeF.vn","")):
        name_company = name_company.replace(*r)
        
    name_company = " ".join(name_company.split())
    name_company = name_company.split(maxsplit=1)[-1]
    return name_company

In [7]:
ck = pd.read_csv('ck1.csv')
ck = ck.replace(np.nan, 1)

In [10]:
def get_folder(index=1):
    ck = pd.read_excel('ck1.xls', sheet_name="1")
    fails = []
    for i in range(241,len(ck)):
        name = ck.loc[i, 'name']
        print(f"Folder {index} - Crawl data {i}: {name}")
        Path(f"data_{index}/{name}").mkdir(parents=True, exist_ok=True)
        
        
        url1 = url.replace("/G36/", f"/{name}/")
        company_name = get_company_name(url1)
        # assign data of lists.  
        data = {'Name': [company_name]}  

        # Create DataFrame  
        df = pd.DataFrame(data)
        df.to_csv(f"data_{index}/{name}/{name}.csv", mode='a')

        print(f"{i}: {name}")
        wrap(url1, name, index)
#         try:
#             wrap(url1, name, index)
#         except:
#             print(f"faile name: {name}")
#             fails.append(name)
#         with open(f"fails_{index}.txt", 'w') as f:
#             for item in fails:
#                 f.write("%s\n" % item)


In [11]:
start = 2017
end = 2021
url = "https://s.cafef.vn/bao-cao-tai-chinh/G36/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn"
get_folder(5)

Folder 5 - Crawl data 241: VSH
241: VSH
Crawl data name VSH
https://s.cafef.vn/bao-cao-tai-chinh/VSH/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/VSH/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/VSH/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/VSH/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/VSH/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 242: OPC
242: OPC
Crawl data name OPC
https://s.cafef.vn/bao-cao-tai-chinh/OPC/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/OPC/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/OPC/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-t

https://s.cafef.vn/bao-cao-tai-chinh/PGD/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PGD/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PGD/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PGD/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 255: TN1
255: TN1
Crawl data name TN1
https://s.cafef.vn/bao-cao-tai-chinh/TN1/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/TN1/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/TN1/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/TN1/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh

https://s.cafef.vn/bao-cao-tai-chinh/CAV/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/CAV/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/CAV/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 268: GMC
268: GMC
Crawl data name GMC
https://s.cafef.vn/bao-cao-tai-chinh/GMC/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/GMC/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/GMC/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/GMC/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/GMC/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 269: SZL
269: 

https://s.cafef.vn/bao-cao-tai-chinh/PAS/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PAS/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 281: PHH
281: PHH
Crawl data name PHH
https://s.cafef.vn/bao-cao-tai-chinh/PHH/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PHH/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PHH/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PHH/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PHH/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 282: G20
282: G20
Crawl data name G20
https://s.cafef.vn/bao-cao-tai-chinh/G20/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-t

https://s.cafef.vn/bao-cao-tai-chinh/VGI/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 294: AAS
294: AAS
Crawl data name AAS
https://s.cafef.vn/bao-cao-tai-chinh/AAS/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/AAS/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/AAS/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/AAS/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/AAS/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 295: NAB
295: NAB
Crawl data name NAB
https://s.cafef.vn/bao-cao-tai-chinh/NAB/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/NAB/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-t

Folder 5 - Crawl data 307: HNM
307: HNM
Crawl data name HNM
https://s.cafef.vn/bao-cao-tai-chinh/HNM/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/HNM/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/HNM/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/HNM/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/HNM/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 308: PGB
308: PGB
Crawl data name PGB
https://s.cafef.vn/bao-cao-tai-chinh/PGB/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PGB/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/PGB/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-t

https://s.cafef.vn/bao-cao-tai-chinh/VNP/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/VNP/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/VNP/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/VNP/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 321: TS4
321: TS4
Crawl data name TS4
https://s.cafef.vn/bao-cao-tai-chinh/TS4/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/TS4/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/TS4/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/TS4/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh

IndexError: list index out of range

In [ ]:
get_folder(2)
get_folder(3)

In [12]:
start = 2017
end = 2021
url = "https://s.cafef.vn/bao-cao-tai-chinh/G36/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn"
fails = []
for i in range(len(ck)):
    name = ck.loc[i,'name']
    Path(f"data_4/{name}").mkdir(parents=True, exist_ok=True)
    
    
    url1 = url.replace("/G36/", f"/{name}/")
    company_name = get_company_name(url1)
    # assign data of lists.  
    data = {'Name': [company_name]}  

    # Create DataFrame  
    df = pd.DataFrame(data)
    df.to_csv(f"data_4/{name}/{name}.csv", mode='a')
    
    print(f"{i}: {name}")
    wrap(url1, name)
#     try:
#         wrap(url1, name)
#     except:
#         print(f"faile name: {name}")
#         fails.append(name)



0: DS3
Crawl data name DS3
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
1: SZB
Crawl data name SZB
https://s.cafef.vn/bao-cao-tai-chinh/SZB/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SZB/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SZB/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SZB/B

In [43]:
for name in fails[2:]:
    url1 = url.replace("/G36/", f"/{name}/")
    company_name = get_company_name(url1)
    # assign data of lists.  
    data = {'Name': [company_name]}  
#     # Create DataFrame  
#     df = pd.DataFrame(data)
#     df.to_csv(f"data/{name}/{name}.csv", mode='a')
#     wrap(url1, name)
#     try:
#         wrap(url1, name)
#     except:
#         print(f"faile name: {name}")
#         fails.append(name)

In [42]:
print(fails)

['NHH', 'VPG', 'VSC', 'HHP', 'HTN', 'KOS', 'ADG', 'VNG', 'THG', 'ASG', 'BIC', 'PGI', 'CLL', 'VNS', 'NHT', 'AST', 'VPS', 'LM8', 'SPP', 'FDG', 'NTB', 'NHP', 'CLG', 'OIL', 'LMH', 'ATA', 'VAB', 'BOT', 'DVN', 'D82', 'HTT', 'NDF', 'ILA', 'MSR', 'VLF', 'HDO', 'AMS', 'SSH', 'PVE', 'APF', 'PRT', 'VNH', 'L44', 'BT6', 'PVA', 'S12', 'STT', 'BTN', 'KLB', 'DGT', 'V15', 'XMC', 'CBI', 'VC5', 'VES', 'PWA', 'AGF', 'NHH', 'VPG', 'VSC', 'HHP', 'HTN', 'KOS', 'ADG', 'VNG', 'VSC', 'VNG', 'THG', 'BIC', 'PGI', 'CLL', 'VNS', 'LM8', 'SPP', 'FDG', 'NTB', 'NHP', 'CLG', 'LMH', 'ATA', 'VAB', 'D82', 'NDF', 'MSR', 'VLF', 'HDO', 'AMS', 'SSH', 'PVE', 'VNH', 'L44', 'BT6', 'PVA', 'S12', 'STT', 'BTN', 'KLB', 'DGT', 'V15', 'VC5', 'VES', 'PWA', 'AGF', 'VSC']


In [68]:
df

,Quý 1-2017,Quý 2-2017,Quý 3-2017,Quý 4-2017,Quý 1-2018,Quý 2-2018,Quý 3-2018,Quý 4-2018,Quý 1-2019,Quý 2-2019,...,Quý 4-2019,Quý 1-2020,Quý 2-2020,Quý 3-2020,Quý 4-2020,Tiêu chí,Quý 1-2021,Quý 2-2021,Quý 3-2021,Quý 4-2021
0,8.200877e+11,8.317475e+11,8.835332e+11,8.482253e+11,8.756774e+11,8.527641e+11,9.200417e+11,8.732920e+11,8.710949e+11,8.377629e+11,...,2.287977e+11,2.291004e+11,0.0,0.0,0.0,I - TÀI SẢN NGẮN HẠN,0.0,0.0,0.0,0.0
1,9.906884e+10,9.964034e+10,9.962368e+10,9.960702e+10,9.959036e+10,9.957370e+10,9.955503e+10,9.953635e+10,9.951767e+10,9.949900e+10,...,9.945542e+10,9.943674e+10,0.0,0.0,0.0,2.3. Tài sản cố định vô hình,0.0,0.0,0.0,0.0
2,1.064663e+12,1.076161e+12,1.157633e+12,1.153313e+12,1.172302e+12,1.166189e+12,1.243079e+12,1.187868e+12,1.217162e+12,1.176087e+12,...,4.254656e+11,4.197661e+11,0.0,0.0,0.0,Tổng cộng tài sản,0.0,0.0,0.0,0.0
3,8.094661e+11,8.199206e+11,8.954360e+11,8.811218e+11,8.980248e+11,8.881622e+11,9.613711e+11,8.942567e+11,9.326025e+11,8.919648e+11,...,8.639636e+11,8.689526e+11,0.0,0.0,0.0,I - NỢ PHẢI TRẢ,0.0,0.0,0.0,0.0
4,7.113610e+11,7.553384e+11,8.751511e+11,8.485322e+11,8.404138e+11,8.322234e+11,9.069384e+11,8.385246e+11,9.068791e+11,8.670724e+11,...,8.472539e+11,8.522429e+11,0.0,0.0,0.0,1. Nợ ngắn hạn,0.0,0.0,0.0,0.0
5,9.810506e+10,6.458222e+10,2.028486e+10,3.258964e+10,5.761095e+10,5.593877e+10,5.443272e+10,5.573210e+10,2.572336e+10,2.489239e+10,...,1.670966e+10,1.670966e+10,0.0,0.0,0.0,2. Nợ dài hạn,0.0,0.0,0.0,0.0
6,2.551973e+11,2.562405e+11,2.621971e+11,2.721915e+11,2.742776e+11,2.780270e+11,2.817080e+11,2.936112e+11,2.845596e+11,2.841225e+11,...,-4.384980e+11,-4.491864e+11,0.0,0.0,0.0,II - VỐN CHỦ SỞ HỮU,0.0,0.0,0.0,0.0
7,1.064663e+12,1.076161e+12,1.157633e+12,1.153313e+12,1.172302e+12,1.166189e+12,1.243079e+12,1.187868e+12,1.217162e+12,1.176087e+12,...,0.000000e+00,4.197661e+11,0.0,0.0,0.0,Tổng cộng nguồn vốn,0.0,0.0,0.0,0.0
8,2.418470e+11,2.581349e+11,3.087644e+11,2.407963e+11,3.061621e+11,2.619465e+11,2.402853e+11,2.983062e+11,1.079823e+11,1.444786e+11,...,1.159238e+09,0.000000e+00,0.0,0.0,0.0,1. Doanh thu bán hàng và cung cấp dịch vụ,0.0,0.0,0.0,0.0
9,2.408171e+11,2.575085e+11,3.085461e+11,2.403129e+11,3.058756e+11,2.611912e+11,2.400551e+11,2.977265e+11,1.079823e+11,1.440040e+11,...,1.060127e+09,0.000000e+00,0.0,0.0,0.0,3. Doanh thu thuần về bán hàng và cung cấp dịc...,0.0,0.0,0.0,0.0


In [65]:
name="SPP"
a = None
url1 = url.replace("/G36/", f"/{name}/")
df = get_head(url1)
df = df.loc[:,~df.columns.duplicated()]
df_eps = get_eps(name)
df_eps=df_eps.replace('-',0)
df_eps = df_eps.loc[:,~df_eps.columns.duplicated()]

https://s.cafef.vn/bao-cao-tai-chinh/SPP/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SPP/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SPP/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SPP/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SPP/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn


In [66]:
df_eps


,CHỈ TIÊU CƠ BẢN,Quý 1/2017,Quý 2/2017,Quý 3/2017,Quý 4/2017,Quý 1/2018,Quý 2/2018,Quý 3/2018,Quý 4/2018,Quý 1/2019,Quý 2/2019,Quý 3/2019,Quý 4/2019,Quý 1/2020
0,EPS,1628,1626,1536,1488,1434,1360,1341,1213,1009,813,326,-28590,-29013


In [67]:
df_eps.shape

(1, 14)

In [76]:
for quarter in range(2,5):

    query = str(quarter) + "-"
    filter_col = [col for col in df if query in col ]
    select_col = ["Tiêu chí"] + filter_col
    df_1 = df[select_col]

    s = df_1.iloc[2]
    n = ["Tổng tải sàn bình quân "]
    n.append(s[1])
    for i in range(2, len(s)):
        n.append(int((s[i]+s[i-1]+1)/2))

    new = pd.Series(n, index=df_1.columns)
    df1 = df_1.append(new, ignore_index=True)
    df2 = df1.iloc[[0,1,2,13,3,4,5,6,7,8,9,10,11,12]]
    df2 = df2.reset_index(drop=True)

    query = str(quarter) + "/"
    filter_col = [col for col in df_eps if query in col ]
    select_col = ["CHỈ TIÊU CƠ BẢN"] + filter_col
    df3 = df_eps[select_col].reset_index(drop=True)

    print(f"{select_col}")
    eps = [df3.iloc[0,0]]
    year = 2016
    for i in range(1,len(df3.columns)):
        year = year + 1
        while str(year) not in df3.columns[i] and year < 2022:
            eps.append(0)
            year = year + 1

        eps.append(int(df3.iloc[0,i]))

    eps = eps + [0] * 5
    k = len(df2.columns)
    eps = eps[0:k]
    df2 = df2.append(pd.Series(eps, index=df2.columns), ignore_index=True)
    print(df2)
    

['CHỈ TIÊU CƠ BẢN', 'Quý 2/2017', 'Quý 2/2018', 'Quý 2/2019']
                                             Tiêu chí    Quý 2-2017  \
0                                I - TÀI SẢN NGẮN HẠN  8.317475e+11   
1                        2.3. Tài sản cố định vô hình  9.964034e+10   
2                                   Tổng cộng tài sản  1.076161e+12   
3                             Tổng tải sàn bình quân   1.076161e+12   
4                                     I - NỢ PHẢI TRẢ  8.199206e+11   
5                                      1. Nợ ngắn hạn  7.553384e+11   
6                                       2. Nợ dài hạn  6.458222e+10   
7                                 II - VỐN CHỦ SỞ HỮU  2.562405e+11   
8                                 Tổng cộng nguồn vốn  1.076161e+12   
9           1. Doanh thu bán hàng và cung cấp dịch vụ  2.581349e+11   
10  3. Doanh thu thuần về bán hàng và cung cấp dịc...  2.575085e+11   
11                                4. Giá vốn hàng bán  2.270891e+11   
12       18. Lợ

In [9]:
print(fails)

['NHH', 'VPG', 'VSC', 'HHP', 'HTN', 'KOS', 'ADG', 'VNG', 'THG', 'ASG', 'BIC', 'PGI', 'CLL', 'VNS', 'NHT', 'AST', 'VPS', 'LM8', 'SPP', 'FDG', 'NTB', 'NHP', 'CLG', 'OIL', 'LMH', 'ATA', 'VAB', 'BOT', 'DVN', 'D82', 'HTT', 'NDF', 'ILA', 'MSR', 'VLF', 'HDO', 'AMS', 'SSH', 'PVE', 'APF', 'PRT', 'VNH', 'L44', 'BT6', 'PVA', 'S12', 'STT', 'BTN', 'KLB', 'DGT', 'V15', 'XMC', 'CBI', 'VC5', 'VES', 'PWA', 'AGF']


In [10]:
len(fails)

57

In [40]:
for i in range(len(ck)):
    name = ck.loc[i,'name']
    print(f"{i}: {name}")
    try:
        Path(f"data/{name}/{name}.csv").unlink()
    except:
        print(f"")
        

0: POM
1: TCM
2: HCD
3: SVT
4: GTN
5: TNT
6: C47
7: TDC
8: HVH
9: VIP
10: ANV
11: CTD
12: SHI
13: LGL
14: VTO
15: FTM
16: TNA
17: BCE
18: NHH
19: PXI
20: ELC
21: VPG
22: VSC
23: PTL

24: GSP

25: SMC

26: PPC

27: TEG

28: ADS

29: GIL

30: HHP

31: NAF

32: CVT

33: HTN

34: HII

35: D2D

36: CTF

37: KOS

38: TDM

39: VRC

40: CNG

41: SJS

42: ASP

43: ADG

44: BWE

45: VNG

46: TV2

47: DBT

48: VID

49: TCO

50: YBM

51: EVE

52: BMC

53: SGR

54: PTB

55: TCR

56: TAC

57: DBD

58: DQC

59: DC4

60: APC

61: DXV

62: HUB

63: HTI

64: THG

65: ILB

66: SFG

67: CHP

68: AAM

69: ASG

70: BIC

71: PGI

72: CMG

73: DVP

74: VMD

75: SBA

76: HU3

77: TLG

78: VPD

79: STG

80: CLL

81: CDC

82: SCS

83: VNS

84: STK

85: NHT

86: GTA

87: SHP

88: AST

89: PJT

90: VPS

91: RAL

92: ST8

93: LM8

94: TCT

95: VFG

96: NAV

97: RIC

98: THI

99: SCD

100: G36

101: PVV

102: SPP

103: FDG

104: TVN

105: NED

106: HLA

107: NTB

108: NHP

109: HKB

110: KHB

111: CLG

112: OIL

113